# CLI

> The project shell entry point—orchestrate all commands

In [ ]:
#| default_exp cli

The CLI module is pure glue code: parse arguments, validate flags, and dispatch to the appropriate function. No business logic lives here—just the translation layer between command-line invocations and the underlying modules.

This separation keeps the CLI thin and the core functions testable. You can call `init_nbdev()`, `sync()`, or `ship()` directly from Python without going through argparse, useful for testing and for potential future GUIs or APIs.

In [ ]:
#| export
import argparse
import sys
from pj.core import kill_processes, get_pj_version
from pj.checks import check_prereqs
from pj.setup import init_nbdev
from pj.sync import sync
from pj.ship import ship

## Argument Parsing

The main entry point constructs the argument parser hierarchy: a root parser with subcommands, each subcommand with its own arguments and flags.

In [ ]:
#| export
def main():
    """Main CLI entry point"""
    parser = argparse.ArgumentParser(
        description=f"""pj > the ProJect shell (v{get_pj_version()})
   | Automate notebook project workflows: init, sync, and ship software.
   | https://kitled.github.io/pj
    """,
        formatter_class=argparse.RawDescriptionHelpFormatter,
        epilog="""
Examples:
  pj init my-project
  pj init my-project -v --desc "My awesome ML library" --private
  pj init my-project --python 3.11 --author "John Doe"
  pj sync
  pj sync -m "Added new feature"
  pj ship
  pj ship --dry-run
  pj kill
        """
    )

    parser.add_argument(
        '--version', '-V',
        action='version',
        version=f'pj {get_pj_version()}'
    )
    
    subparsers = parser.add_subparsers(dest="command", help="Available commands")
    
    # init subcommand
    init_parser = subparsers.add_parser(
        "init",
        help="Initialize a new nbdev project",
        formatter_class=argparse.RawDescriptionHelpFormatter,
        description="""
Initialize a new nbdev project with all the bells and whistles:
- GitHub repository creation
- nbdev project structure with Jupyter hooks
- Virtual environment with uv
- Jupyter kernel registration
- direnv auto-activation
        """,
        epilog="""
Examples:
  pj init my-project
  pj init my-project -v --desc "ML utilities" --private
  pj init my-project --python 3.11 --license mit
        """
    )
    
    # Required arguments
    init_parser.add_argument("name", help="Project name (will be repo and package name)")
    
    # Output options
    output_group = init_parser.add_argument_group("output options")
    output_group.add_argument(
        "-v", "--verbose",
        action="store_true",
        help="Show detailed command output"
    )
    output_group.add_argument(
        "--logfile",
        help="Path to log file (default: PROJECT/init.log)"
    )
    output_group.add_argument(
        "--no-log",
        action="store_true",
        help="Disable logging to file"
    )
    
    # GitHub options
    gh_group = init_parser.add_argument_group("GitHub options")
    gh_group.add_argument(
        "--org",
        help="Create repository under this organization (default: personal account)"
    )
    gh_group.add_argument(
        "--public",
        action="store_true",
        help="Create public repository (default: private)"
    )
    gh_group.add_argument(
        "--description", "--desc",
        help="Repository description"
    )
    
    # Python/venv options
    py_group = init_parser.add_argument_group("Python options")
    py_group.add_argument(
        "--python",
        help="Python version (e.g., 3.11, 3.12)"
    )
    
    # nbdev options
    nbdev_group = init_parser.add_argument_group("nbdev options")
    nbdev_group.add_argument(
        "--author",
        help="Author name (default: from git config)"
    )
    nbdev_group.add_argument(
        "--author-email",
        dest="author_email",
        help="Author email (default: from git config)"
    )
    nbdev_group.add_argument(
        "--license",
        choices=["apache2", "mit", "gpl3", "bsd3"],
        help="License type (default: apache2)"
    )
    nbdev_group.add_argument(
        "--min-python",
        dest="min_python",
        help="Minimum Python version (default: 3.9)"
    )
    
    # Post-init options
    post_group = init_parser.add_argument_group("post-init options")
    post_group.add_argument(
        "--no-preview",
        action="store_true",
        help="Skip opening nbdev_preview"
    )
    post_group.add_argument(
        "--no-lab",
        action="store_true",
        help="Skip launching Jupyter Lab"
    )
    post_group.add_argument(
        "-c", "--code",
        action="store_true",
        help="Open VSCode"
    )
    post_group.add_argument(
        "-q", "--quiet",
        action="store_true",
        help="Quiet mode: skip preview and lab (just cd + tree)"
    )

    # sync subcommand
    sync_parser = subparsers.add_parser(
        "sync",
        help="Sync project: pull, prepare, and push to GitHub",
        description="""
Sync your nbdev project in one command:
1. git pull (aborts on merge conflicts)
2. nbdev_prepare (export, test, clean - aborts if tests fail)
3. git commit -am "message"
4. git push
        """,
        epilog="""
Examples:
  pj sync                    # Uses default message "save"
  pj sync -m "Added tests"   # Custom commit message
  pj sync -v                 # Verbose output
        """
    )
    sync_parser.add_argument(
        "--message", "-m",
        help="Commit message (default: 'save')"
    )
    sync_parser.add_argument(
        "-v", "--verbose",
        action="store_true",
        help="Show detailed command output"
    )

    # ship subcommand
    ship_parser = subparsers.add_parser(
        "ship",
        help="Ship a new release: bump version, build, upload, tag, and release",
        description="""
Ship a complete release in one command:
1. Check for uncommitted changes
2. Bump version with nbdev_bump_version
3. Commit and push version bump
4. Build and upload to PyPI with nbdev_pypi
5. Create git tag and push
6. Create GitHub release with auto-generated notes
        """,
        epilog="""
Examples:
  pj ship                    # Bump patch version (0.0.X)
  pj ship --part 1           # Bump minor version (0.X.0)
  pj ship --part 0           # Bump major version (X.0.0)
  pj ship --dry-run          # Preview without making changes
  pj ship --skip-pypi        # Skip PyPI upload (for testing)
        """
    )
    ship_parser.add_argument(
        "--part",
        type=int,
        default=2,
        choices=[0, 1, 2],
        help="Version part to bump: 0=major, 1=minor, 2=patch (default: 2)"
    )
    ship_parser.add_argument(
        "--dry-run",
        action="store_true",
        help="Show what would be done without making changes"
    )
    ship_parser.add_argument(
        "--force",
        action="store_true",
        help="Ship even with uncommitted changes (not recommended)"
    )
    ship_parser.add_argument(
        "--skip-pypi",
        action="store_true",
        help="Skip PyPI upload (for testing)"
    )
    ship_parser.add_argument(
        "--skip-gh-release",
        action="store_true",
        help="Skip GitHub release creation"
    )
    ship_parser.add_argument(
        "-v", "--verbose",
        action="store_true",
        help="Show detailed command output"
    )

    # kill subcommand
    kill_parser = subparsers.add_parser(
        "kill",
        help="Kill all running background processes (jupyter, nbdev_preview, quarto)"
    )
    
    args = parser.parse_args()
    
    if args.command == "init":
        init_nbdev(args)
    elif args.command == "sync":
        sync(args)
    elif args.command == "ship":
        ship(args)
    elif args.command == "kill":
        kill_processes(args)
    else:
        parser.print_help()
        sys.exit(1)

### CLI Architecture

**Subcommand dispatch**

Argparse's `subparsers` creates a command hierarchy: `pj <command> [options]`. Each subcommand gets its own parser with specific arguments. The `dest="command"` attribute stores which subcommand was invoked, making dispatch trivial: check `args.command` and call the appropriate function.

**Argument groups**

The `init` subcommand has many flags. Organizing them into logical groups (`output_group`, `gh_group`, `py_group`, etc.) structures the help text. Users see related options clustered together rather than one long alphabetized list.

**Raw description formatting**

`RawDescriptionHelpFormatter` preserves whitespace and newlines in description and epilog strings. This lets us write multi-line examples and numbered workflows that display exactly as written, rather than getting reflowed into paragraphs.

**Default values**

Many arguments have defaults specified in the business logic functions, not here. The CLI passes `None` for optional arguments, and the called function provides the default. This keeps defaults in one place—changing the default license doesn't require updating both CLI and init logic.

**The dispatch pattern**

The final `if/elif` chain is the entire dispatch logic. Parse args, check which command, call the function, pass the args object. No validation, no transformation—the called functions handle everything. This keeps the CLI dumb and the modules smart.

**No command behavior**

If the user runs `pj` with no subcommand, we print the help and exit with code 1. This is more helpful than an error message—the user sees all available commands immediately.

### Future: Additional Commands

**TODO: `pj dev` command**

Launch development servers (Jupyter Lab, nbdev_preview) using the project's venv jupyter. Should replace the post-init server launching currently inline in `init_nbdev()`.

**TODO: `pj kernel` command**

Manage Jupyter kernels: list installed kernels, uninstall by name, re-register after moving projects. Common operations that currently require remembering `jupyter kernelspec` syntax.

**TODO: `pj check` command**

Run just the prerequisite checks without initializing a project. Useful for verifying your system is ready before starting a batch of projects.

These extensions are natural once the core workflow is solid. The modular structure makes adding new commands straightforward: create the function in an appropriate module, add the subparser here, dispatch to the function.